### 챔피언 승리,패배 크롤링

In [1]:
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

service = Service(executable_path="../../Text_mining/chromedriver.exe")
options = webdriver.ChromeOptions()

### 픽밴 분석

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
request_headers = { 
'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0;Win64; x64)\
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98\
Safari/537.36'), } 

In [13]:
url = f'https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category=&category2=181&shipcode=&shipgroup=&teamName=&startDate=&endDate='
html_ = requests.get(url, headers = request_headers).text    
banPic = pd.read_html(html_)[-1]


In [14]:
banPic.drop('순위',axis=1, inplace=True)
banPic.sort_values('밴+픽', ascending=False, inplace=True)

In [16]:
banPic.to_csv('밴픽.csv',encoding='utf-8', index=False)

In [ ]:
# 젠지 : 1792
# t1 : 1791
# kt : 1770
# 디플러스 : 1765
# 징동 : 1776
# blg : 1785
# lng : 1786
# 웨이보 : 1774
# 지투 : 1816
# 프나틱 : 1815
# bds : 2033
# 매드라이온즈 : 1817
# c9 : 1794
# 팀리키드 : 1801
# nrg : 2096
# gam esports : 1862

### 팀 능력치 정보

In [5]:
team_dic = {'Gen.G eSports': 1792, 'KT Rolster':1770,'Dplus KIA':1765, 'T1':1791,
            'JD Gaming':1776,'Bilibili Gaming':1785, 'LNG Esports':1786, 'Weibo Gaming':1774, 
            'G2 Esports':1816,'Fnatic':1815,'Team BDS':2033,'MAD Lions':1817,
            'Cloud9':1794,'Team Liquid':1801,'NRG':2096, 'GAM Esports':1862}

In [6]:
driver = webdriver.Chrome()
teamStat = []

for key,val in tqdm(team_dic.items()):
    url = f'https://gol.gg/teams/team-stats/{val}/split-ALL/tournament-ALL/'
    driver.get(url)
    page_source = driver.page_source
    
    dd = pd.read_html(page_source)[7].iloc[[0,1,8]]
    dd['Economy.1'] = dd['Economy.1'].str.replace('+','').astype('float')
    time.sleep(np.random.random())
    dd['Name'] = key
    teamStat.append(dd)
ts = pd.concat(teamStat)
ts.to_csv('팀능력.csv',encoding='utf-8', index=False)   

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:36<00:00,  2.31s/it]


### 팀 챔피언 선택 정보

In [64]:
gameList = []
teamList = []
champList = []
driver = webdriver.Chrome()

for key,val in tqdm(team_dic.items()):
    url = f'https://gol.gg/teams/team-stats/{val}/split-ALL/tournament-ALL/'
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    gameNum = soup.find('div',class_= 'row rowbreak pb-4').find('tbody').find_all('span', class_='text-center') 
    
    for i in gameNum:
        gameList.append(i.text.split()[-1])
        teamList.append(key)
        champList.append(i.find('strong').text)
    time.sleep(2.5+np.random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [01:36<00:00,  6.01s/it]


In [65]:
gt = pd.DataFrame({'team':teamList, 'champ':champList,'pick':gameList})
gt.to_csv('팀챔피언분석.csv',encoding='utf-8', index=False)    